# Código para Verificar o Arquivo Xml por Erros

In [2]:
# -*- coding: utf-8 -*-
"""
Spyder Editor

This is a temporary script file.
"""

import xml.etree.cElementTree as ET
import pprint
from collections import defaultdict

#função para contagem de tags presentes
def count_tags(filename):
    dicionario=defaultdict(int)
    for event, elem in ET.iterparse(filename):
        dicionario[elem.tag]+=1
    return dicionario


#analisar as tags presentes
tags = count_tags('GrandeFortaleza.osm')
pprint.pprint(tags)


defaultdict(<type 'int'>, {'node': 220814, 'nd': 302069, 'bounds': 1, 'member': 6949, 'tag': 139721, 'relation': 1526, 'way': 44496, 'osm': 1})


o osm é simplesmente a linha que define a versão do mapa e bounds é o definidor da região que estou a analisar. way, node e relation são os 3 tipos basicos 'nd' são os nós definidores de uma way. tags são características da tag superior. e a member define os membros de uma relação. Logo as tags estão ok.

In [3]:
import re
normal = re.compile(r'^([a-z]|_)*$',re.IGNORECASE)
normal_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$',re.IGNORECASE)
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
uppercase_incorrect=re.compile("[A-Z][A-Z]+")

problem=[]
other=[]
upper=[]
def key_type(element, keys):
    if element.tag == "tag":
        if  uppercase_incorrect.search(element.attrib["k"]):
            upper.append(element.attrib["k"])
            keys["upper"]+=1
        if  normal.match(element.attrib["k"])!=None:
            keys["normal"]+=1
        elif  normal_colon.match(element.attrib["k"])!=None:
            keys["normal_colon"]+=1
        elif  problemchars.search(element.attrib["k"])!=None:
            keys["problemchars"]+=1
            problem.append(element.attrib["k"])
        else:
            keys["other"]+=1
            other.append(element.attrib["k"])
    return keys



def process_map(filename):
    keys = {"normal": 0, "normal_colon": 0, "problemchars": 0, "other": 0, "upper": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

print (process_map('GrandeFortaleza.osm'))

{'problemchars': 213, 'normal_colon': 25923, 'other': 228, 'upper': 1354, 'normal': 113357}


tivemos uma boa quantidade de caractéres mínusculos com ou sem dois pontos. Eles já foram trabalhados nos exercícios e num primeiro momento estão ok. Irei investigar os problemchars e os other agora.

In [4]:
    pprint.pprint(problem[:10])

['Testada 2',
 'Profundidade 2',
 'Testada 2',
 'Profundidade 2',
 'Testada 2',
 'Profundidade 2',
 'Testada 1',
 'Testada 2',
 'Profundidade 1',
 'Profundidade 2']


Como foi somente em alguns acredito que possa ser consertado. grande parte é só espaço vazio e uma abreviação simples Unid.Hab.


já para o outro caso.

In [5]:
    pprint.pprint(other[:10])

['seamark:light:range',
 'seamark:light:colour',
 'seamark:light:height',
 'seamark:light:period',
 'seamark:light:character',
 'seamark:light:reference',
 'seamark:light:range',
 'seamark:light:colour',
 'seamark:light:height',
 'seamark:light:character']


Os com 2 :  grande parte dos casos vem de coisas como o seamark que são tipos especificos de chaves.

In [6]:
pprint.pprint(upper[:15])

['IBGE:GEOCODIGO',
 'IBGE:GEOCODIGO',
 'IBGE:GEOCODIGO',
 'IBGE:GEOCODIGO',
 'IBGE:GEOCODIGO',
 'IBGE:GEOCODIGO',
 'IBGE:GEOCODIGO',
 'IBGE:GEOCODIGO',
 'IBGE:GEOCODIGO',
 'IBGE:GEOCODIGO',
 'UNIDADES HABITACIONAIS',
 'FID',
 'FID',
 'FID',
 'FID']


grande parte é só transformar maisculas e minúsculas. a APROJETADA tem que ser transformado em Área Projetada também há siglas como FID que tem que ser descoberto o sentido e transformado ou não nele.

Analizarei prelinariamente agora o código postal

In [14]:
count=0
for _, element in ET.iterparse('GrandeFortaleza.osm'):
    for elem in element.iter('tag'):
        if elem.attrib['k']=='addr:postcode' or elem.attrib['k']=='postcode':
            print elem.attrib
            count+=1
    if(count==20):
        break

{'k': 'addr:postcode', 'v': '60115-280'}
{'k': 'addr:postcode', 'v': '60115-280'}
{'k': 'addr:postcode', 'v': '60115-080'}
{'k': 'addr:postcode', 'v': '60115-080'}
{'k': 'addr:postcode', 'v': '60160-150'}
{'k': 'addr:postcode', 'v': '60160-150'}
{'k': 'addr:postcode', 'v': '60150-161'}
{'k': 'addr:postcode', 'v': '60150-161'}
{'k': 'addr:postcode', 'v': '60060-130'}
{'k': 'addr:postcode', 'v': '60060-130'}
{'k': 'addr:postcode', 'v': '60165070'}
{'k': 'addr:postcode', 'v': '60165070'}
{'k': 'addr:postcode', 'v': '60020-110'}
{'k': 'addr:postcode', 'v': '60020-110'}
{'k': 'addr:postcode', 'v': '60822-131'}
{'k': 'addr:postcode', 'v': '60822-131'}
{'k': 'addr:postcode', 'v': '60822-530'}
{'k': 'addr:postcode', 'v': '60822-530'}
{'k': 'addr:postcode', 'v': '60140-100'}
{'k': 'addr:postcode', 'v': '60140-100'}


encontrei tanto no formato com hífem quanto sem hífem. irei retirar o hífem na análise. Os resultados foram curiosamente poucos.

analisarei agora o id

In [9]:
count1=0
unicos=set()
for _, element in ET.iterparse('GrandeFortaleza.osm'):
    if(element.tag=='node' or element.tag=='way'):
        count1+=1
        unicos.add(element.attrib["id"])
print count1,len(unicos)

265310 265310


todas as id são únicas

Para terminar olhar os dados com o nome 'fixme'

In [15]:
count=0
for _, element in ET.iterparse('GrandeFortaleza.osm'):
    for elem in element.iter('tag'):
        if elem.attrib['k']=='fixme':
            print elem.attrib
            count+=1
    if(count==20):
        break            

{'k': 'fixme', 'v': u'provavelmente est\xe1 com tag errada'}
{'k': 'fixme', 'v': u'provavelmente est\xe1 com tag errada'}
{'k': 'fixme', 'v': u'Dado coletado h\xe1 muito tempo, aparentemente mudou o nome, mas permanece uma bicicletaria..'}
{'k': 'fixme', 'v': u'Dado coletado h\xe1 muito tempo, aparentemente mudou o nome, mas permanece uma bicicletaria..'}
{'k': 'fixme', 'v': 'nome da rua parece errado, talvez seja um nome local'}
{'k': 'fixme', 'v': 'nome da rua parece errado, talvez seja um nome local'}
{'k': 'fixme', 'v': 'melhorar posicionamento'}
{'k': 'fixme', 'v': 'melhorar posicionamento'}
{'k': 'fixme', 'v': 'limite aparentemente no local errado'}
{'k': 'fixme', 'v': 'limite aparentemente no local errado'}
{'k': 'fixme', 'v': u'Ajustar contorno \xe0 \xe1rea onde predominam moradias vizinhas.'}
{'k': 'fixme', 'v': u'Ajustar contorno \xe0 \xe1rea onde predominam moradias vizinhas.'}
{'k': 'fixme', 'v': u'Ajustar contorno \xe0 \xe1rea onde predominam moradias vizinhas. Melhorar no

Para terminar vou verificar o nome das ruas.

In [16]:
count=0
for _, element in ET.iterparse('GrandeFortaleza.osm'):
    for elem in element.iter('tag'):
        if (elem.attrib['k'] == "addr:street"):
            print elem.attrib
            count+=1
    if(count==20):
        break

{'k': 'addr:street', 'v': 'Rua Nestor Barbosa'}
{'k': 'addr:street', 'v': 'Rua Nestor Barbosa'}
{'k': 'addr:street', 'v': 'Rua Padre Anchieta'}
{'k': 'addr:street', 'v': 'Rua Padre Anchieta'}
{'k': 'addr:street', 'v': 'Avenida Santos Dumont'}
{'k': 'addr:street', 'v': 'Avenida Santos Dumont'}
{'k': 'addr:street', 'v': u'Avenida Bar\xe3o de Studart'}
{'k': 'addr:street', 'v': u'Avenida Bar\xe3o de Studart'}
{'k': 'addr:street', 'v': u'Rua Visconde de Mau\xe1'}
{'k': 'addr:street', 'v': u'Rua Visconde de Mau\xe1'}
{'k': 'addr:street', 'v': u'Rua Maria Tom\xe1sia'}
{'k': 'addr:street', 'v': u'Rua Maria Tom\xe1sia'}
{'k': 'addr:street', 'v': u'Rua Doutor Jos\xe9 Louren\xe7o'}
{'k': 'addr:street', 'v': u'Rua Doutor Jos\xe9 Louren\xe7o'}
{'k': 'addr:street', 'v': u'Avenida Bar\xe3o de Studart'}
{'k': 'addr:street', 'v': u'Avenida Bar\xe3o de Studart'}
{'k': 'addr:street', 'v': u'Avenida Bar\xe3o de Studart'}
{'k': 'addr:street', 'v': u'Avenida Bar\xe3o de Studart'}
{'k': 'addr:street', 'v': 

analisando no olho não notei erros. Logo irei fazer uma procura pelos tipos de rua.

In [12]:
street_type_re = re.compile(r'^\b\S+\.?', re.IGNORECASE)
expected = ["Rua","Avenida","Beco","Alameda","Acesso","Travessa", "CE"]

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
            
def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if (tag.attrib['k'] == "addr:street"):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types
st_types = audit('GrandeFortaleza.osm')
pprint.pprint(dict(st_types))

{'2': set(['2']),
 'AV': set(['AV JOAQUIM LOPES DE ABREU']),
 'Alphaville': set(['Alphaville Fortaleza']),
 u'Alu\xedsio': set([u'Alu\xedsio Mamede']),
 'Av': set([u'Av Santa Cec\xedlia', 'Av Washington Soares']),
 'Av.': set(['Av. Beira-Mar',
             'Av. Dr. Almir Pinto',
             u'Av. Her\xf3is do Acre',
             'Av. Leite Barbosa',
             'Av. Washington Soares',
             u'Av. Zez\xe9 Diogo']),
 'Barbosa': set(['Barbosa de Freitas']),
 'Campus': set(['Campus do Pici', 'Campus do Pici - UFC']),
 'Costa': set(['Costa Barros']),
 u'Eng\xb0': set([u'Eng\xb0 Sandoval S\xe1']),
 'Joaquim': set(['Joaquim Nabuco']),
 u'Jos\xe9': set([u'Jos\xe9 Severino']),
 u'Jo\xe3o': set([u'Jo\xe3o Paula Costa']),
 'Maria': set(['Maria Teixeira Joca']),
 u'Pra\xe7a': set([u'Pra\xe7a Pelotas']),
 u'R.': set([u'R. Prof. Jos\xe9 Henrique da Silva',
             u'R. Santa Rita das Causas Imposs\xedveis']),
 'Santa': set(['Santa Teresinha']),
 'Titan': set(['Titan']),
 'Tv.': set(['

Ao contrário da auditoria no olho foi detectado vários problemas nas ruas também

In [21]:
teste = re.compile(r'[\,]', re.IGNORECASE)
for _, element in ET.iterparse('GrandeFortaleza.osm'):
    for elem in element.iter('tag'):
        if (teste.search(elem.attrib['k'])!=None ) :
            print elem.attrib['k']

Não há vírgulas no arquivo. Logo é seguro usar elas como o limitador no csv.